In [2]:
%pip install pedalboard
%pip install python-dotenv
%load_ext dotenv
%dotenv



The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv



In [3]:
from dotenv import dotenv_values
config = dotenv_values(".env") 

In [4]:
pluginsDir = config['PLUGIN_HOME']
tracksDir = config['TRACKS_HOME']
mixDir = config['MIX_HOME']

In [5]:
from pedalboard import load_plugin

proc2_path = pluginsDir + 'FabFilter Pro-C 2.vst3'
prog_path = pluginsDir + 'FabFilter Pro-G.vst3'

def config_c2(numOfLevels, currentLevel, c2):
  c2.knee = (72 / numOfLevels) * currentLevel
  c2.range =  1
  c2.auto_release = True
  c2.release = 100
  c2.attack = 33

proC2 = load_plugin(proc2_path)

In [6]:
def config_prog(prog): 
  prog.threshold = -45
  prog.attack = 0
  prog.hold = 77
  prog.knee = 8
  prog.release = 1.0
  prog.lookahead_enabled = True
  prog.lookahead = 1
  prog.left_side_chain_mix = -1
  prog.right_side_chain_mix = 1
  return prog

  
prog = config_prog(load_plugin(prog_path))

In [7]:
import numpy as np

def normalize(audio):
    max_val = np.max(np.abs(audio))
    if max_val > 0:
        normalized = audio / max_val
    return normalized

def normalize2(audio, n):
    normalized = audio / n
    return normalized

def bounce(audios):
  bounce = audios[0]
  for i in range (1, len(audios)):
    bounce = bounce + audios[i]
  return [normalize(bounce), normalize(normalize2(bounce, len(audios)))]

In [8]:
depth_config = [
  ['LV-STEM.flac'],
  ['BV STEM.flac'],
  ['PIANO STEM.flac'], 
  ['GTR STEM.flac'],
  ['KICK STEM.flac'],
  ['KIT STEM.flac'],
  ['PERC STEM.flac'], 
  ['BASS STEM.flac', 'SNARE STEM.flac', 'LV-STEM.flac'],
]

In [9]:
from pedalboard.io import AudioFile
from pedalboard import Pedalboard
import numpy as np
import os

def mix_files(depth_config):
  outputs = []
  inputs = []
  levels = len(depth_config)
  max_samples = 0
  samplerate = 44100
  for i in range(0, levels):
    print('processing depth ' + str(i+1))
    songs_at_depth = depth_config[i]
    for j in range(0, len(songs_at_depth)):
      file_path = tracksDir + songs_at_depth[j]
      print('Processing ' + songs_at_depth[j])
      f = AudioFile(file_path)
      audio = f.read(f.frames)
      samplerate = f.samplerate
      max_samples = max(max_samples, audio.shape[1])
      inputs.append(audio)
      
      config_c2(levels, i, proC2)
      print(' Samplerate is ' + str(samplerate))
      board = Pedalboard([prog, proC2])
      outputs.append(board(audio, samplerate, reset=False))
      print('Processed ' + songs_at_depth[j])
  print('adding songs');
  for i in range(len(inputs)):
    shape = inputs[i].shape
    if shape[1] < max_samples:
      if (shape[0] == 1):
        inputs[i] = np.pad(inputs[i], (0, max_samples - shape[1]), 'constant')
        outputs[i] = np.pad(outputs[i], (0, max_samples - shape[1]), 'constant')
      else:
        inputs[i] = np.pad(inputs[i], ((0,0), (0, max_samples - shape[1])), 'constant')
        outputs[i] = np.pad(outputs[i], ((0,0), (0, max_samples - shape[1])), 'constant')
  print('normalizing songs');
  # Normalize the mixed audio to prevent clipping
  mixed_audio = bounce(outputs)
  pre_mixed_audio = bounce(inputs)

  # Save the output
  output_file = mixDir + 'mixed_output.wav'
  output_file3 = mixDir + 'mixed_output2.wav'
  output_file2 = mixDir + 'pre_mixed_output.wav'
  output_file4 = mixDir + 'pre_mixed_output2.wav'

  print('saving songs');
  with AudioFile(output_file, 'w', samplerate, outputs[0].shape[0]) as f:
      f.write(mixed_audio[0])
  with AudioFile(output_file3, 'w', samplerate, outputs[0].shape[0]) as f:
      f.write(mixed_audio[1])
  with AudioFile(output_file2, 'w', samplerate, outputs[0].shape[0]) as f:
      f.write(pre_mixed_audio[0])
  with AudioFile(output_file4, 'w', samplerate, outputs[0].shape[0]) as f:
      f.write(pre_mixed_audio[1])
  print('done');
  return [output_file, output_file2, output_file3, output_file4]

In [10]:
mix_files(depth_config)

processing depth 1
Processing LV-STEM.flac
 Samplerate is 44100
Processed LV-STEM.flac
processing depth 2
Processing BV STEM.flac
 Samplerate is 44100
Processed BV STEM.flac
processing depth 3
Processing PIANO STEM.flac
 Samplerate is 44100
Processed PIANO STEM.flac
processing depth 4
Processing GTR STEM.flac
 Samplerate is 44100
Processed GTR STEM.flac
processing depth 5
Processing KICK STEM.flac
 Samplerate is 44100
Processed KICK STEM.flac
processing depth 6
Processing KIT STEM.flac
 Samplerate is 44100
Processed KIT STEM.flac
processing depth 7
Processing PERC STEM.flac
 Samplerate is 44100
Processed PERC STEM.flac
processing depth 8
Processing BASS STEM.flac
 Samplerate is 44100
Processed BASS STEM.flac
Processing SNARE STEM.flac
 Samplerate is 44100
Processed SNARE STEM.flac
Processing LV-STEM.flac
 Samplerate is 44100
Processed LV-STEM.flac
adding songs
normalizing songs
saving songs
done


['./.mixes/mixed_output.wav',
 './.mixes/pre_mixed_output.wav',
 './.mixes/mixed_output2.wav',
 './.mixes/pre_mixed_output2.wav']